<a href="https://colab.research.google.com/github/flake313/Malashin/blob/main/2_MATRIX_LAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

# Коэффициенты системы уравнений
a = [
    [10.0, 2.0, 1.0, 14.0],  # 10x + 2y + 1z = 14
    [1.0, 10.0, 2.0, 15.0],  # 1x + 10y + 2z = 15
    [2.0, 1.0, 10.0, 18.0]   # 2x + 1y + 10z = 18
]

# Функция для вычисления детерминанта с частичным выбором главного элемента
def determinant(matrix):
    n = len(matrix)
    det = 1
    swap_sign = 1  # Для отслеживания количества обменов строк

    for i in range(n):
        # Поиск максимального элемента в текущем столбце для частичного выбора главного элемента
        max_row = i
        for k in range(i + 1, n):
            if abs(matrix[k][i]) > abs(matrix[max_row][i]):
                max_row = k

        # Обмен строк для частичного выбора главного элемента
        if i != max_row:
            matrix[i], matrix[max_row] = matrix[max_row], matrix[i]
            swap_sign *= -1  # Каждый обмен строк меняет знак детерминанта

        # Если на диагонали оказался нулевой элемент, детерминант равен нулю
        if matrix[i][i] == 0:
            return 0

        det *= matrix[i][i]

        # Приведение элементов ниже диагонали к нулю
        for k in range(i + 1, n):
            factor = matrix[k][i] / matrix[i][i]
            for j in range(i, n):
                matrix[k][j] -= factor * matrix[i][j]

    return det * swap_sign

# Функция для решения системы методом Крамера
def solve_cramer(matrix, b, eps=1e-10):
    n = len(b)
    # Создаем копию матрицы для избежания изменения исходной
    det_A = determinant([row[:] for row in matrix])

    if np.isclose(det_A, 0, atol=eps):
        raise ValueError("Детерминант равен нулю или близок к нулю. Решение невозможно (Метод Крамера).")

    # Решение системы
    x = np.zeros(n)

    # Перебираем все столбцы
    for i in range(n):
        temp_matrix = [row[:] for row in matrix]  # Создаем копию матрицы для изменения

        # Заменяем столбец матрицы на столбец свободных членов
        for j in range(n):
            temp_matrix[j][i] = b[j]

        det_temp = determinant(temp_matrix)

        if np.isclose(det_temp, 0, atol=eps):
            raise ValueError(f"Детерминант модифицированной матрицы равен нулю (Cramer) для x[{i}].")

        # Решение по формуле x[i] = det(A_i) / det(A)
        x[i] = det_temp / det_A

    return x

# Проверка диагонального преобладания
def is_diagonally_dominant(a):
    for i in range(len(a)):
        sum_other_elements = sum(abs(a[i][j]) for j in range(len(a[i]) - 1) if j != i)
        if abs(a[i][i]) <= sum_other_elements:
            return False
    return True

# Метод Якоби (итерационный метод)
def jacobi_method(a, tolerance=1e-6, max_iterations=1000):
    n = len(a)
    x = [0.0 for _ in range(n)]
    x_new = x[:]

    for _ in range(max_iterations):
        for i in range(n):
            s = sum(a[i][j] * x[j] for j in range(n) if j != i)
            x_new[i] = (a[i][n] - s) / a[i][i]

        if all(abs(x_new[i] - x[i]) <= tolerance for i in range(n)):
            return x_new
        x = x_new[:]

    return x_new

# Основная программа
try:
    print("Проверка метода Крамера...")
    # Вектор свободных членов (последний столбец)
    b = [row[3] for row in a]
    # Матрица коэффициентов (первые 3 столбца)
    matrix = [row[:3] for row in a]
    result_cramer = solve_cramer(matrix, b)
    print(f"Метод Крамера: x = {result_cramer[0]}, y = {result_cramer[1]}, z = {result_cramer[2]}")
except ValueError as e:
    print(e)

# Если система не является диагонально доминирующей, метод Якоби может не сойтись
if is_diagonally_dominant(a):
    result_jacobi = jacobi_method(a)
    print("Метод Якоби:")
    print(f"x = {result_jacobi[0]}, y = {result_jacobi[1]}, z = {result_jacobi[2]}")
else:
    print("Система не является диагонально доминирующей, метод Якоби может не сходиться.")


Проверка метода Крамера...
Метод Крамера: x = 1.0316122233930451, y = 1.1001053740779767, z = 1.4836670179135931
Метод Якоби:
x = 1.0316124155344, y = 1.1001055662422998, z = 1.483667210017
